# Corner Minor Families of Paths
A matrix $A\in\R^{n\times n}$ is oscillatory if it is I-TN and there exists a postive integer $q$ such that $A^q$ is TP. The least positive integer $r=r(A)$ such that $A^r$ is TP is called the *exponent* of the oscillatory matrix $A$. Determining the exponent of an oscillatory matrix is an important problem.

Here, we implemented few tools that can be used to determine the exponent of an oscillatory matrix.

Given an oscillatory matrix $A=LDU$, where $L$ [$U$] denotes the product of the left-hand side [right-hand side] of the SEB factorization of $A$, we implement the computation of the family of paths corresponding to all lower-left [upper-right] corner minors of $L$ [$U$]. The paths corresponding to $U$ are determined by computing the paths of all lower-left corner minors of $U^T$ and then converting them to the paths of all upper-right corner minors of $U$.
Specifically, for each lower-left corner minor, we compute the minimum number of copies of $A$ such that a family of vertex-disjoint paths can be realized. In addition, we display the corresponding family for each corner minor, for both $L$ and $U$.

The code here uses a triangle graph that is derived from the planar network of the SEB factorization of I-TN matrices. For more information see the paper: [On the Exponent of Several Classes of Oscillatory Matrices](https://arxiv.org/abs/1910.10709)

## Functions and Classes

In [7]:
from collections import defaultdict
import numpy as np
import copy 
import re
import TP_TN_OSC_funcs as tpf  # generated by eb_factorization_example.ipynb

'''
Functions for lower-left corner minor paths.
=============================================
We use the triangle graph (or table) of the left-hand side of the SEB factorization of a I-TN matrix.

Below is an example of a triangle graph for n=5. Here "O" indicates the existance
of the corresponding L_i matrix, and X the absent. The indexes on the left indicates the source
indexes in a lower-left minor index set, and the indexes on the bottom the destination set.
In general for n=5, the first column in the triangle graph corresponts to L_5L_4L_3L_2, the second to L_5L_4L_3, 
the third to L_5L_4 and the last column to L_5. In each column, the top sign (i.e. "O" or "X") 
correspondds to L_n in L_nL_{n-1}....L_i. 

For example, the triangle graph below corresponds to [L_5L_3L_2][L_5L_3][L_4][L_5].

    5 O
    4 X O
    3 O X X
    2 O O O O
      1 2 3 4

A path in the triangle graph is defined by a sequence of 2D point along the graph. We 
start at the source indexes on the left, and move either down or diagonally. 
Moving along a diagonal in the triangle graph corresponds to horizontal path in the planar network. 
Moving down in the graph corresponds to moving down along a diagonal in the planar network.

A path in the triangle graph is represented by a list of 2D points (which are lists by themself
of two numbers, the y-axis value and the x-axis value). 
A path is constructed as follows: We move down if we are at a "O", or diagonally if we are at "X".
We stop once we reach the destination.


For example, the path that starts in the source index 5 (i.e. in the 2D point [5,1]) in the
graph above and ends in the destination index 3 (i.e. in the 2D point [1,3]) is:
 
 [[5,1],[4,1],[3,2],[2,3],[1,3]]. 

Note that in this case, this is the minimal index that the 
source index 5 ([5,1]) can reach in the graph. This path correponds to the following steps:

1. starting from [5,1] corresponds to starting at the "O" in the top sign in the first column.
2. moving down to the "X" sign below it (i.e. to [4,1]) corresponds to using L_5 in the [L_5L_3L_2] 
   (in the planar network).
3. moving diagonally to the "X" in the second column (i.e. to [3,2]) corresponds to using an horizontal edge
   (in the planar network).
4. moving diagonally to the "X" in the third column (i.e. to [2,3]) corresponds to using an horizontal edge
   (in the planar network).
5. moving down to [1,3] correponds to using L_4 in [L_4]. At that point we reached the destination index. 

Given a 2D point in the triangle graph [q_y, q_x] the corresponding index in the planar network 
is q_y_q_x-1. Thus, the path above corresponds to the following path in the planar network:

  [5, 4, 4, 4, 3].
  
The code below implements these steps. In cases where a destination can not be reached (for example, 
in the example above [5,1] (i.e. index 5) can not reach index [1,1] (i.e. index 1)) an additional
copy of the triangle graph is used. This is repeated until a path is realized.

In the context an oscillatory matrix A=LDU, we use the triangle graph to compute the families of
vertex-disjoint paths corresponding to all lower-left corner minors of L and U^T, and thus deduce
the exponent of the matrix.
'''


def valsL2L_info(valsL: list) -> list:
    '''This function generates L_info given the
    L_i parameter values corresponding to [L_n*...*L_2]*[L_n*...*L_3]*...*[Ln].
    
    L_info is a list of lists. Each list element in L_info contains all the i indexes
    for which L_i exists in the EB factorization. The first list element in info
    corresponds to [L_n,...,L_2], the second to [L_n,...,L_3], etc.
    
    For example, for n=4 and left-hand side factorization [L5_L_4L_2][L_5L_3][L_5L_4][],
    the corresponding L_info is [[5,4,2],[4,3],[5,4],[]].
    
    See the function EB_factorization_ITN() in TP_TN_OSC_funcs.py that 
    generates valsL given an I-TN matrix A = LDU.
    Note that valsL here can also be equal to valsU that is also generated by 
    EB_factorization_ITN() in order to generate the L_info of U^T.
    '''
    n = tpf.EB_factorization_k2n(len(valsL))
    all_L = list(tpf.compute_L_indexes(n))
    cms = np.r_[[0], np.cumsum([x for x in range(n-1,0,-1)])]  # [0 cumsum([n-1,...,1])]
    L_info = []
    for i in range(n-1):
        g = all_L[cms[i]:cms[i+1]]      # the corresponding section in the factorization
        v = valsL[cms[i]:cms[i+1]] > 0  # the corresponding L_i with parameter>0
        L_info.append([k for (k, f) in zip(g, v) if f])
    return(L_info)

def init_tri_tbl(L_info: list):
    '''This function initializes a triangle table based on the left-hand side
    of the SEB factorization.
    
    The returned numpy array contains the True/False triangle table located 
    upside-down in its upper-left side. All the other elements in the returned
    table (i.e. elements outside the upside-down triangle) are set to False.
    
    For example, for L_info = [[5,4,2],[4,3],[5,4],[]], the returned array is:
    
    [[ True  True  True False]
     [False  True  True False]
     [ True False False False]
     [ True False False False]]
     
     i.e. the upside-down True/False triangle table is:
     
      True  True  True  False
     False  True  True
      True False 
      True
      
      which correspondonds to the (not upside-down) triangle table:
      
      O
      O X
      X O O 
      O O O O
    '''
    n1 = len(L_info)
    tbl = np.zeros((n1,n1), dtype=bool)

    for x in range(n1):
        for y in range(n1+1,x+1,-1):
            '''we check if n,n-1,...,x+2 exists in L_info[x],
            where x = 0,1,...,n1-1'''
            tbl[y-2-x,x] = y in L_info[x] 
    return tbl

def view_tri_tbl(tri_tbl: list, pre: str ='', lb: dict={True: 'O', False: 'X'}, verbose: bool=True, dig_width: int=2, num_space: int=1) -> None:
    '''This function prints the triangle graph tri_tbl.'''
    show, s = 1, ' '
    for y in range(tri_tbl.shape[0]-1, -1, -1):
        print(pre, end='')
        if verbose:
            print(f"{y+2:{dig_width}}{s*num_space}", end='')
        for x in range(show):
            print(f"{lb[tri_tbl[y,x]]}", end=s*num_space)
        show += 1
        print()
    if verbose:
        print(f"{pre}{s*(num_space+dig_width)}", end='')
        for i in range(tri_tbl.shape[0]):
            print(f"{i+1}", end=s*num_space)
        print()

        
def gen_LL_minor_indexes(n: int, x: int):
    '''This function generates a lower-left minor indexes. 
    The indexes in the minor are n-x+1,...,n|1,...,x. 
    This is returned as a numpy array np.array([[n-x+1,...,n], [1,...,x]])
    '''
    return np.array([[i for i in range(n-x+1,n+1)],[i for i in range(1,x+1)]])

def minor_indexes2str(mnr) -> str:
    '''This function converts the numpy 2D array of the minor indexes to a string
    "s|d", where s contains the source indexes and d the destination indexes.
    For example minor_indexes2str([[5,6],[1,2]]) results in '5,6|1,2'.
    '''
    return f"{','.join(map(str, mnr[0,:]))}|{','.join(map(str, mnr[1,:]))}"

def index_path(tf_tbl, d: int, oy: int=1, ox: int=0) -> list:
    '''This function computes the path of a source index to its destination
    index in the triangle graph. d here is the destination (1D) index.
    The start 2D index in the graph is (y,x)=(tf_tbl.shape[0],1) and we move 
    along the triangle graph either down (if we can, i.e.
    if there is no X) or diagonally (in which case we increase 
    the x axis index). We stop once we reach a destination (i.e. 
    if (y+1)+x=1+d).
    '''
    x, p = 1, []
    for y in range(tf_tbl.shape[0],0,-1):
        p.append([y,x])
        '''We move diagonally (corresponkding to horizontal
        path in the planar network) if:
        1. We encountered X (i.e. False), or
        2. We reached the destination 2D index. A destination 2D index
           is [1,d], and we reached a destination index if (y+1)+x = 1+d.
           Recall that here y values are: n-1,n-2,...,1, thus we need to add
           1 to y to get the real 2D index.'''
        if (not tf_tbl[y-1,x-1]) or (x+y==d):
            x += 1  
            
    # convert to indexes corresponding to the triangle graph.
    return path_offset(p+[[y-1,x]], oy, ox)      


def update_tf_tbl(po: list, tri_tbl) -> None:
    '''This function adds X (i.e. False) in the 2D indexes that are above the given path po. 
    An index above (y,x) is defined by (y+1,x). 
    Note that in po, y take values from 2,3,...,n, and x take values from 1,2,...,n-1.
    Thus, in the numpy triangle table tri_tbl (where y values are 0,1,...,) we need to 
    decrease y by 1 in order to mark the 2D index above the corresponding y indexes in po.
    '''
    for y,x in po:
        tri_tbl[y-1,x-1] = False  # the 2D index above the indexes in the path po
        #tri_tbl[y-2,x-1] = False # the indexes in the path po

def tf_tbl_trig_slice(tf_tbl, y: int):
    '''Given a value y (y \in\{2,3,...,n\}), the function returns the triangle sub-table
    whos top-left 2D index is (y,1).'''
    return tf_tbl[0:y-1,0:y-1]

def path_offset(p: list, oy: int, ox: int=0) -> list:
    '''This function offsets the y and x indexes in the path p.'''
    po = copy.deepcopy(p)
    for i in range(len(p)):
        po[i][0] += oy
        po[i][1] += ox
    return po

def path_2to1(path:list) ->list:
    '''This function converts a 2D representation of a path to 1D representation.
    For example path_2to1([[5,1],[4,2],[3,2],[2,2],[1,3]]) returns [5,5,4,3,3]'''
    return [x+y-1 for (x,y) in path]

def show_1dpath(path1d: list, flip=None, compress=None) -> str:
    '''This function returns a string in the form x1->x1->...x_n, 
    for path1d = [x1,x2,...,xn]. If flip not None then
    the returned string is xn->...->x1.
    If compress not None, output is x1-->xn (or xn-->x1)'''
    flip = False if flip is None else True
    compress = False if compress is None else True

    p = path1d.copy()[::-1] if flip else path1d
    s = ''
    if compress:
        s = f"{p[0]}-->"
    else:
        if len(p)==1:
            s = f"{p[0]}->" 
        for x in p[:-1]:
            s += f"{x}->"
    return s+f"{p[-1]}"

def horizontal_path(idx: int) -> list:
    '''This function returns a horizontal path in the triangle graph.
       For example, if idx=3, then the horizontal path is [[3,1], [2,2], [1,3]].
       '''
    return [[idx-i,i+1] for i in range(idx)]
    
def flip_str_by_sp(llm: str, sp: str) -> str:
    '''Given a string input the function flips the string around a separator (sp).
    For example flip_str_by_sp("3,4,5|1,2,3", "|") yields "1,2,3|3,4,5".
    '''
    z = re.match(f"(.+)\{sp}(.+)", llm)
    return f"{z.group(2)}{sp}{z.group(1)}"
    
def run_ll_minor(mnr, mnr_tbl, verbose: bool=False, MAX_copies: int=100) -> tuple:
    '''This function computes the paths of a lower-left minor.
    Inputs: 
       mnr: a lower-left minor indexes, i.e. n-x+1,...,n|1,...,x, in the
            form of a numpy array [[n-x+1,...,n],[1,...,x]]
       mnr_tbl: the triangle table corresponding to the minor indexes.
       
    Outputs:
       num_copies - minimum number of matrix copies in order to obtain a single family of
                    vertex-disjoint paths.
       paths - a dictionary of paths, where keys are source->destination indexes.
    '''
    tf_tbl = np.copy(mnr_tbl)
    mnr_str = minor_indexes2str(mnr)
    done, num_copies = False, 1
    cur_mnr = mnr.copy()
    num_idxs = mnr.shape[1]  # number of indexes in the minor
    dest = np.zeros(num_idxs, dtype=int)  # container for destination of each index
    idxs_done = [False for i in range(num_idxs)]
    paths = defaultdict(list)
    vprint = print if verbose else lambda *a, **k: None
    
    vprint(f"\nMinor {mnr_str}:\n-----------------------")
    while not done:
        vprint(f'\n\tCopy {num_copies}, current graph:')
        view_tri_tbl(mnr_tbl, '\t') if verbose else None
        for k in range(num_idxs):
            cur_idxs = cur_mnr[:,k]
            vprint(f"\n\tPath for {cur_idxs[0]}->{cur_idxs[1]}...")
            if cur_idxs[0] > cur_idxs[1]: 
                # have yet to reach the destination vertex, i.e. source > destination
                sl_tbl = tf_tbl_trig_slice(mnr_tbl, cur_idxs[0])
                vprint(f"\tCorresponding graph:")
                view_tri_tbl(sl_tbl, '\t') if verbose else None
                po = index_path(sl_tbl, cur_idxs[1])
                if k < num_idxs - 1: # no need to update the table after the last index
                    update_tf_tbl(po, mnr_tbl)
            else:
                '''use horizontal path as we are already in the destination. For
                example, if cur_indxs[0]=cur_indxs[1]=3, then the horizontal path
                is [[3,1], [2,2], [1,3]] '''
                po = horizontal_path(cur_idxs[1])
            vprint(f"\t{po=}")
            if (d:=po[-1][1]) == cur_idxs[1]:
                vprint(f"\tDone.")
                idxs_done[k] = True
            else:
                vprint(f"\tFinished at {d}, need at least one more copy...")

            dest[k]=d
            paths[f"{mnr[0][k]}->{mnr[1][k]}"].append(po)
        
        if not all(idxs_done):
            # prepare for another copy of the triangle table
            mnr_tbl = tf_tbl_trig_slice(np.copy(tf_tbl), dest[-1])
            num_copies += 1
            cur_mnr[0,:] = np.array(dest) # the sources of the paths for the next copy are [dest[0],...,dest[-1]]
        else:
            done = True
        if num_copies >= MAX_copies:
            print(f"Reached {MAX_copies} copies, aborting...")
            done = True
    return paths, num_copies  # note that num_copies = len(paths[k]), for any key k.

'''
Classes
==========
'''

class T_graph():
    '''
    The triangle graph class corresponding to an I-TN matrix or  
    information of two left-hand side of SEB factorization.
    '''
    A = None   # Matrix A that is I-TN can be represented by two triangle graphs.
    L_info: list = [] 
    UT_info: list = []
    tf_tbls: list = []
    n: int = 0
    
    mat_names: dict = {"L":"L", "UT":"U^T"} 
    
    def __init__(self, A=None, L_info: list=None, UT_info: list=None) -> None:
        if A is None:
            print(f"{self.__class__.__name__}: Using L_info and UT_info to initialize object")
            self.L_info = L_info
            self.UT_info = UT_info
            self.n = len(L_info)+1
        else:
            print(f"{self.__class__.__name__}: Using A to initialize object")
            self.A = A
            _, _, _, _, valsL, valsU  = tpf.EB_factorization_ITN( A )  
            self.L_info = valsL2L_info(valsL)
            self.UT_info = valsL2L_info(valsU)
            self.n = A.shape[0]
            
        self.tf_tbls = {self.mat_names["L"]: init_tri_tbl(self.L_info), self.mat_names["UT"]: init_tri_tbl(self.UT_info)}
        
    def set_mat_names(self, mnames: dict) -> None:
        self.mat_names = mnames
        
    def show_trig_tbls(self, pre: str='') -> None:
        for k in self.tf_tbls.keys():
            if self.tf_tbls[k].size > 0:
                print(f"Triangle graph associated with {k}:")
                view_tri_tbl(self.tf_tbls[k], pre)
            else:
                print(f"No triangle graph associated with {k}.")


class LLC_minors():
    '''
    The lower-left corner minors class.
    '''
    ll_mnr_indx: list = []
    
    vd_families: dict = defaultdict(dict)  # results for L and U^T
    num_copies: dict = defaultdict(int)
    Uvd_families: dict = defaultdict(dict)  # results for L and U
    Unum_copies: dict = defaultdict(int)
    
    def __init__(self, t_graph):
        self.t_graph = t_graph
        self.ll_mnr_indx = [i+1 for i in range(self.t_graph.n-1)]

    def mat_names(self) -> str:
        return self.t_graph.mat_names
    
    def display_raw_results(self) -> None:
        print(f"\nResults:\n-----------------")
        print("Number of copies:")
        pprint.pprint(dict(self.num_copies))
        print("Paths:")
        pprint.pprint(dict(self.vd_families))
        r_L_UT = []
        for k in self.num_copies.keys():
            r_L_UT.append(max(self.num_copies[k].values()))
            print(f"Exponent of {k}: {r_L_UT[-1]}.")
        print(f"Matrix exponent = {max(r_L_UT)}")

    def display_results(self, compress=None) -> None:
        '''Displays the results for L and U^T.'''
        print(f"\nResults for L and U^T:\n----------------------")
        for k in self.num_copies.keys():
            print(f"{k} (r({k})={max(self.num_copies[k].values())})")
            for m in self.num_copies[k].keys():
                print(f"\t{m}", end=' ')
                print(f"({self.num_copies[k][m]} copies):") # number of copies
                for p in self.vd_families[k][m].keys():
                    s = ''.join([f"[{show_1dpath(path_2to1(q), compress=compress)}]" for q in self.vd_families[k][m][p]])
                    print(f"\t\t{s}")
        print(f"Matrix exponent = {max([max(self.num_copies[k].values()) for k in self.num_copies.keys()])}")
 
    def display_LU_results(self, compress=None) -> None:
        '''Displays the results for L and U'''
        self.UT2U_results() # convert U^T results to U results
        print(f"\nResults for L and U:\n----------------------")
        for k in self.Unum_copies.keys():
            print(f"{k} (r({k})={max(self.Unum_copies[k].values())})")
            for m in self.Unum_copies[k].keys():
                print(f"\t{m}", end=' ')
                print(f"({self.Unum_copies[k][m]} copies):") # number of copies
                for p in self.Uvd_families[k][m].keys():
                    s = ''.join([f"[{show_1dpath(path_2to1(q), compress=compress)}]" for q in self.Uvd_families[k][m][p]])
                    print(f"\t\t{s}")
        print(f"Matrix exponent = {max([max(self.num_copies[k].values()) for k in self.num_copies.keys()])}")
 

    def UT2U_results(self, name: str = None) -> None:
        '''This function converts the results of U^T to the results of U.
        We simply flip each minor indexes and flip the paths'''
        name = "U" if name is None else name
        
        # copy results
        self.Unum_copies[self.mat_names()["L"]] = self.num_copies[self.mat_names()["L"]]
        nrsl = self.num_copies[self.mat_names()["UT"]]
        self.Unum_copies[name] = {flip_str_by_sp(k, sp='|'): v for (k,v) in nrsl.items()}
        
        #Unrsl = defaultdict(int)
        #for (k,v) in nrsl.items():
        #    Unrsl[flip_str_by_sp(k, sp='|')] = v
        #self.Unum_copies[name] = dict(Unrsl)
        
        # Paths results
        self.Uvd_families[self.mat_names()["L"]] = self.vd_families[self.mat_names()["L"]]
        rsl = self.vd_families[self.mat_names()["UT"]]
        Ursl = defaultdict(dict)
        for k in rsl.keys():  # keys here are the minors
            #kr = flip_str_by_sp(k, sp='|') 
            #prsl = defaultdict(dict)
            '''
            we reverse the order here since in a upper-right corner minor
            we start the paths with the maximal index (as oppose to the case of
            a lower-left corner minor where we start with the minimal index).
            '''
            #for pi in sorted(rsl[k].keys(), reverse=True):
            #    rpi = flip_str_by_sp(pi, sp='->') # flip source-destination indexes
                
                #pths = []
                #for p in rsl[k][pi][::-1]: # reverse order of copies
                #    pths.append(p[::-1])
                    #print(p[::-1])
                #prsl[rpi] = pths
                #prsl[rpi] = [p[::-1] for p in rsl[k][pi][::-1]]
            #Ursl[kr] = dict(prsl)
            Ursl[flip_str_by_sp(k, sp='|')] = {flip_str_by_sp(pi, sp='->'): [p[::-1] for p in rsl[k][pi][::-1]] for pi in sorted(rsl[k].keys(), reverse=True)}
           
        self.Uvd_families[name] = dict(Ursl)
        
        
    def run(self, verbose: bool=True) -> list:
        '''This function computes the paths and number of copies.'''
        for c, Q in enumerate(self.mat_names().values()):  # processing L and U^T for A = LDU. 
            tf_tbl = self.t_graph.tf_tbls[Q]
            if verbose:
                print(f"\nProcessing the matrix {Q}:\n==========================") 
                print(f'Triangle table:') 
                view_tri_tbl(tf_tbl)
                
            # containers per minor
            mnrs_num_copies = defaultdict(int)
            mnrs_paths = defaultdict(dict)
            
            # Path analysis for a minor 
            for l in self.ll_mnr_indx:
                mnr_tbl = np.copy(tf_tbl)
                mnr = gen_LL_minor_indexes(self.t_graph.n, l)
                mnr_str = minor_indexes2str(mnr)
            
                # computes paths per minor
                paths, ncopies = run_ll_minor(mnr, mnr_tbl)
                
                if verbose:
                    print(f"\n{mnr_str}: {ncopies} copies. Paths:")
                    for k in paths.keys():
                        print(f"{k}:")
                        for i in range(len(paths[k])):
                            print(f"\tCopy {i+1}: {paths[k][i]}")  
                mnrs_num_copies[mnr_str] = ncopies
                mnrs_paths[mnr_str] = dict(paths)
                
            self.vd_families[Q] = dict(mnrs_paths)
            self.num_copies[Q] = dict(mnrs_num_copies)
                
        return [self.num_copies, self.vd_families]   
                

## An Example

In [8]:
import pprint
import numpy as np

# Inputs
# ------
'''
The inputs are:
1. An I-TN matrix A, or (if A is not defined),
2. A list of L_is of L and U^T.
'''

#L_info = [[6,5,4,3,2],[],[],[],[]]
#UT_info = [[6,5,4,3,2],[6,5,4,3],[6,5,4],[6,5],[6]] 

# all L_i are present in L
L_info = [[5,4,3,2],[5,4,3],[5,4],[5]] 

'''L_4 and L_3 are missing in [L_5L_4L_3L_2], L_5 and L_4 are missing in [L_5L_4L_3]
L5 is missing from [L_5L_4] and L_5 is missing from [L5]
'''
UT_info = [[5,2], [3], [4], []]  
A = None #np.array([[3, 1, 0, 0], [1, 4, 1, 0.1], [0.1, 1, 5, 3], [0, 0, 2, 7]])
# ----------------------------------------------------------------------------------

# first create a graph object
G = T_graph(L_info=L_info, UT_info=UT_info, A=A)
# display triangle graphs for L and U^T
G.show_trig_tbls()
# create a lower-left corner minor object based on the graph object
LC = LLC_minors(G)
# run the paths computation
LC.run(verbose=False)
# display results for L and U^T
LC.display_results()
# or we can display results for L and U
LC.display_LU_results()
# display raw results
#LC.display_raw_results()

T_graph: Using L_info and UT_info to initialize object
Triangle graph associated with L:
 5 O 
 4 O O 
 3 O O O 
 2 O O O O 
   1 2 3 4 
Triangle graph associated with U^T:
 5 O 
 4 X X 
 3 X X X 
 2 O O O X 
   1 2 3 4 

Results for L and U^T:
----------------------
L (r(L)=1)
	5|1 (1 copies):
		[5->4->3->2->1]
	4,5|1,2 (1 copies):
		[4->3->2->1]
		[5->5->4->3->2]
	3,4,5|1,2,3 (1 copies):
		[3->2->1]
		[4->4->3->2]
		[5->5->5->4->3]
	2,3,4,5|1,2,3,4 (1 copies):
		[2->1]
		[3->3->2]
		[4->4->4->3]
		[5->5->5->5->4]
U^T (r(U^T)=3)
	5|1 (3 copies):
		[5->4->4->4->3][3->3->2][2->1]
	4,5|1,2 (3 copies):
		[4->4->4->3][3->3->2][2->1]
		[5->5->5->5->5][5->4->4->4->3][3->3->2]
	3,4,5|1,2,3 (2 copies):
		[3->3->2][2->1]
		[4->4->4->3][3->3->2]
		[5->5->5->5->5][5->4->4->4->3]
	2,3,4,5|1,2,3,4 (2 copies):
		[2->1][1->1]
		[3->3->2][2->2]
		[4->4->4->3][3->3->3]
		[5->5->5->5->5][5->4->4->4->4]
Matrix exponent = 3

Results for L and U:
----------------------
L (r(L)=1)
	5|1 (1 copies):
		[5->4->